In [77]:
!pip install -r requirements.txt

# Document QA

In [78]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.indexes.vectorstore import VectorstoreIndexCreator
import os


os.environ["OPENAI_API_KEY"] = 'Enter your api key here'

# Prepare Data

In [79]:
from langchain.document_loaders import TextLoader

# loader = TextLoader("./state_of_the_union.txt", encoding="utf-8")
# state_of_the_union = loader.load()
# print(state_of_the_union)

with open("./state_of_the_union.txt",encoding="utf-8") as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

embeddings = OpenAIEmbeddings()

In [80]:
docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))]).as_retriever()

In [81]:
query = "What did the president say about Justice Breyer"
docs = docsearch.get_relevant_documents(query)

# Quickstart

If you just want to get started as quickly as possible, this is the recommended way to do it:

In [82]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [83]:
chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff")
query = "What did the president say about Justice Breyer"
chain.run(input_documents=docs, question=query)

' The president said that Justice Breyer is an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court, and thanked him for his service.'

# The Stuff Chain


The stuff chain is a simple way to combine multiple documents together and feed them into a language model. It works best when the documents are short and only a few of them are used at a time.

In [84]:
chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff")

In [85]:
query = "What did the president say about Justice Breyer"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': ' The president said that Justice Breyer is an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court, and thanked him for his service.'}

## Custom Prompts

You can also use your own prompts with this chain. In this example, we will respond in Italian.

In [86]:
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer in Italian:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff", prompt=PROMPT)
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': ' Il presidente ha detto che Justice Breyer ha dedicato la sua vita al servizio di questo paese e ha ringraziato per il suo servizio.'}

# The map_reduce Chain


The map reduce process involves two steps: mapping and reducing. First, each document goes through a series of actions called the LLM chain individually (map). The output of these actions is treated as a new document. Then, all the new documents are combined together to create a final result (reduce). If needed, the mapped documents can be compressed to fit in the combine step, and this compression can be done repeatedly if necessary.

In [87]:
chain = load_qa_chain(OpenAI(temperature=0), chain_type="map_reduce")

In [88]:
query = "What did the president say about Justice Breyer"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': ' The president said that Justice Breyer is an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court, and thanked him for his service.'}

# Intermediate Steps

We can also return the intermediate steps for map_reduce chains, should we want to inspect them. This is done with the return_map_steps variable.

In [89]:
chain = load_qa_chain(OpenAI(temperature=0), chain_type="map_reduce", return_map_steps=True)

In [90]:
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'intermediate_steps': [' "Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service."',
  ' A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans.',
  ' None',
  ' None'],
 'output_text': ' The president said that Justice Breyer is an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court, and thanked him for his service.'}

# Custom Prompts

You can also use your own prompts with this chain. In this example, we will respond in Italian.

In [91]:
question_prompt_template = """Use the following portion of a long document to see if any of the text is relevant to answer the question. 
Return any relevant text translated into italian.
{context}
Question: {question}
Relevant text, if any, in Italian:"""
QUESTION_PROMPT = PromptTemplate(
    template=question_prompt_template, input_variables=["context", "question"]
)

combine_prompt_template = """Given the following extracted parts of a long document and a question, create a final answer italian. 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.

QUESTION: {question}
=========
{summaries}
=========
Answer in Italian:"""
COMBINE_PROMPT = PromptTemplate(
    template=combine_prompt_template, input_variables=["summaries", "question"]
)
chain = load_qa_chain(OpenAI(temperature=0), chain_type="map_reduce", return_map_steps=True, question_prompt=QUESTION_PROMPT, combine_prompt=COMBINE_PROMPT)
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'intermediate_steps': ["\nStasera vorrei onorare qualcuno che ha dedicato la sua vita a servire questo paese: il giustizia Stephen Breyer - un veterano dell'esercito, uno studioso costituzionale e un giustizia in uscita della Corte Suprema degli Stati Uniti. Giustizia Breyer, grazie per il tuo servizio.",
  '\nNessun testo pertinente.',
  " Non c'è testo pertinente per rispondere alla domanda.",
  ' Non ha detto nulla riguardo a Justice Breyer.'],
 'output_text': ' Non ha detto nulla riguardo a Justice Breyer.'}

# Batch Size

When using the map_reduce chain, one thing to keep in mind is the batch size you are using during the map step. If this is too high, it could cause rate limiting errors. You can control this by setting the batch size on the LLM used. Note that this only applies for LLMs with this parameter. Below is an example of doing so:

In [92]:
llm = OpenAI(batch_size=5, temperature=0)

# The refine Chain

The refine documents chain uses a looping process to update its answer by analyzing each document one at a time. This method is good for tasks that involve many documents but has the drawback of making more calls to the model. It may not work well when documents refer to each other or when detailed information is needed from multiple documents.

In [93]:
chain = load_qa_chain(OpenAI(temperature=0), chain_type="refine")

In [94]:
query = "What did the president say about Justice Breyer"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': '\n\nThe president said that he wanted to honor Justice Breyer for his dedication to serving the country, his legacy of excellence, and his commitment to advancing liberty and justice, while also emphasizing the importance of competition and the need to go after criminals who have stolen relief money meant for small businesses and millions of Americans. He also expressed his support for the LGBTQ+ community and the need to pass the Equality Act, as well as his commitment to strengthening the Violence Against Women Act. Additionally, he offered a Unity Agenda for the Nation to beat the opioid epidemic.'}

# Intermediate Steps

We can also return the intermediate steps for refine chains, should we want to inspect them. This is done with the return_refine_steps variable.

In [95]:
chain = load_qa_chain(OpenAI(temperature=0), chain_type="refine", return_refine_steps=True)

In [96]:
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'intermediate_steps': ['\nThe president said that he wanted to honor Justice Breyer for his dedication to serving the country and his legacy of excellence.',
  '\nThe president said that he wanted to honor Justice Breyer for his dedication to serving the country, his legacy of excellence, and his commitment to advancing liberty and justice.',
  '\nThe president said that he wanted to honor Justice Breyer for his dedication to serving the country, his legacy of excellence, and his commitment to advancing liberty and justice, while also emphasizing the importance of competition and the need to go after criminals who have stolen relief money meant for small businesses and millions of Americans.',
  '\n\nThe president said that he wanted to honor Justice Breyer for his dedication to serving the country, his legacy of excellence, and his commitment to advancing liberty and justice, while also emphasizing the importance of competition and the need to go after criminals who have stolen relie

# Custom Prompts

You can also use your own prompts with this chain. In this example, we will respond in Italian.

In [97]:
refine_prompt_template = (
    "The original question is as follows: {question}\n"
    "We have provided an existing answer: {existing_answer}\n"
    "We have the opportunity to refine the existing answer"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{context_str}\n"
    "------------\n"
    "Given the new context, refine the original answer to better "
    "answer the question. "
    "If the context isn't useful, return the original answer. Reply in Italian."
)
refine_prompt = PromptTemplate(
    input_variables=["question", "existing_answer", "context_str"],
    template=refine_prompt_template,
)


initial_qa_template = (
    "Context information is below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {question}\nYour answer should be in Italian.\n"
)
initial_qa_prompt = PromptTemplate(
    input_variables=["context_str", "question"], template=initial_qa_template
)
chain = load_qa_chain(OpenAI(temperature=0), chain_type="refine", return_refine_steps=True,
                     question_prompt=initial_qa_prompt, refine_prompt=refine_prompt)
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'intermediate_steps': ['\nIl presidente ha detto che Justice Breyer ha dedicato la sua vita al servizio di questo paese e ha reso omaggio al suo servizio.',
  "\nIl presidente ha detto che Justice Breyer ha dedicato la sua vita al servizio di questo paese, ha ricevuto un ampio sostegno da parte della Fraternal Order of Police e di ex giudici nominati da democratici e repubblicani, e ha reso omaggio al suo servizio. Ha anche sottolineato l'importanza di avanzare la libertà e la giustizia, sia attraverso la sicurezza delle frontiere che attraverso la risoluzione del sistema di immigrazione.",
  "\n\nIl presidente ha detto che Justice Breyer ha dedicato la sua vita al servizio di questo paese, ha ricevuto un ampio sostegno da parte della Fraternal Order of Police e di ex giudici nominati da democratici e repubblicani, e ha reso omaggio al suo servizio. Ha anche sottolineato l'importanza di avanzare la libertà e la giustizia, sia attraverso la sicurezza delle frontiere che attraverso la r

# The map-rerank Chain

The map re-rank documents chain uses a prompt to evaluate documents and find the best answer, considering both the completion of a task and the certainty of the response. It returns the response with the highest score.

In [98]:
chain = load_qa_chain(OpenAI(temperature=0), chain_type="map_rerank", return_intermediate_steps=True)

In [99]:
query = "What did the president say about Justice Breyer"
results = chain({"input_documents": docs, "question": query}, return_only_outputs=True)

c:\Users\Mr Darklord\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain\chains\llm.py:303: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [100]:
results["output_text"]

' The President thanked Justice Breyer for his service and honored him for dedicating his life to serve the country.'

In [101]:
results["intermediate_steps"]

[{'answer': ' The President thanked Justice Breyer for his service and honored him for dedicating his life to serve the country.',
  'score': '100'},
 {'answer': ' This document does not answer the question', 'score': '0'},
 {'answer': ' This document does not answer the question', 'score': '0'},
 {'answer': ' This document does not answer the question', 'score': '0'}]

# Custom Prompts

You can also use your own prompts with this chain. In this example, we will respond in Italian.

In [102]:
from langchain.output_parsers import RegexParser

output_parser = RegexParser(
    regex=r"(.*?)\nScore: (.*)",
    output_keys=["answer", "score"],
)

prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

In addition to giving an answer, also return a score of how fully it answered the user's question. This should be in the following format:

Question: [question here]
Helpful Answer In Italian: [answer here]
Score: [score between 0 and 100]

Begin!

Context:
---------
{context}
---------
Question: {question}
Helpful Answer In Italian:"""
PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"],
    output_parser=output_parser,
)

chain = load_qa_chain(OpenAI(temperature=0), chain_type="map_rerank", return_intermediate_steps=True, prompt=PROMPT)
query = "What did the president say about Justice Breyer"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'intermediate_steps': [{'answer': ' Il presidente ha detto che Justice Breyer ha dedicato la sua vita a servire questo paese.',
   'score': '100'},
  {'answer': " Il presidente ha detto che Justice Breyer è un costruttore di consenso che ha ricevuto un ampio sostegno da parte dell'Ordine Fraterno della Polizia e di ex giudici nominati da democratici e repubblicani.",
   'score': '50'},
  {'answer': ' Non so.', 'score': '0'},
  {'answer': ' Non so.', 'score': '0'}],
 'output_text': ' Il presidente ha detto che Justice Breyer ha dedicato la sua vita a servire questo paese.'}

# Document QA with sources

We can also perform document QA and return the sources that were used to answer the question. To do this we'll just need to make sure each document has a "source" key in the metadata, and we'll use the load_qa_with_sources helper to construct our chain:

In [103]:
docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))])
query = "What did the president say about Justice Breyer"
docs = docsearch.similarity_search(query)

In [104]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")
query = "What did the president say about Justice Breyer"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': " The president thanked Justice Breyer for his service and mentioned that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to continue Justice Breyer's legacy of excellence.\nSOURCES: 31-pl"}